# Check on the accuracy of the calculation of the QOCT gradients: case with two perturbation fields

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
#from qutip import *
import qutip as qt

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In order to calculate the optimal pulse that induces a given reaction in a quantum system, one defines a function of that pulse that must be optimized. One important ingredient for the optimization is derivative of this function with respect to the control parameters that define the pulse. In this script we check that this gradient or derivative is calculated correctly.

## Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

# The target is the population of the first excited state.
target_level = 1

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
nperts = 2
H_m1 = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H_m2 = np.array([0, 0, 0.001], dtype = float)
H0 = GdW30.hGdW30(D, E, H)
V1 = GdW30.vGdW30(H_m1)
V2 = GdW30.vGdW30(H_m2)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates) - eigenvalues[0]
V1 = V1.transform(eigenstates)
V2 = V2.transform(eigenstates)

In [ ]:
H = hamiltonians.hamiltonian(H0, [V1, V2])

In [ ]:
w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i]
    taui[i] = 2.0*np.pi/w[i]
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

## Time array definition

In [ ]:
T = 5*taui[0]
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 2.0)
print('# Time steps =', time.shape[0])
print('# Delta t =', time[1])

## Control functions

The control function are parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
M1 = 10
M2 = 8

u1 = np.zeros(2*M1+1)
u1[2] = 1.0
u1[3] = 1.0

u2 = np.zeros(2*M2+1)
u2[1] = 1.0

f1 = pulses.pulse("fourier", T, u = u1)
f2 = pulses.pulse("fourier", T, u = u2)

In [ ]:
fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), f1.fu(time))
ax.plot(time * 1000/(2.0*np.pi), f2.fu(time))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# Gradient calculation for state-to-state transitions

## QOCT target function definition

In [ ]:
lambda_ = 0.1 / sp.integrate.simps(f1.fu(time)*f1.fu(time), time)

def Pfunction(u):
    f1.set_parameters(u[0:f1.nu])
    f2.set_parameters(u[f1.nu:f1.nu+f2.nu])
    return - lambda_ * sp.integrate.simps(f1.fu(time) * f1.fu(time), time) \
           - lambda_ * sp.integrate.simps(f2.fu(time) * f2.fu(time), time)

def Fyu(y, u):
    x = qt.expect(qt.fock_dm(dim, target_level), y)
    return x + Pfunction(u)

def Pfunction(u):
    f1.set_parameters(u[0:f1.nu])
    f2.set_parameters(u[f1.nu:f1.nu+f2.nu])
    return - lambda_ * sp.integrate.simps(f1.fu(time) * f1.fu(time), time) \
           - lambda_ * sp.integrate.simps(f2.fu(time) * f2.fu(time), time)

def dPdu(u, m):
    f1.set_parameters(u[0:f1.nu])
    f2.set_parameters(u[f1.nu:f1.nu+f2.nu])
    if m < f1.nu:
        return - 2.0 * lambda_ * sp.integrate.simps(f1.dfu(time, m) * f1.fu(time), time)
    else:
        return - 2.0 * lambda_ * sp.integrate.simps(f2.dfu(time, m-f1.nu) * f2.fu(time), time)

#def dFdu(u, m):
#    f1.set_parameters(u[0:f1.nu])
#    f2.set_parameters(u[f1.nu:f1.nu+f2.nu])
#    if m < f1.nu:
#        return - 2.0 * lambda_ * sp.integrate.simps(f1.dfu(time, m) * f1.fu(time), time)
#    else:
#        return - 2.0 * lambda_ * sp.integrate.simps(f2.dfu(time, m-f1.nu) * f2.fu(time), time)

def dFdy(y, u):
    return qt.fock_dm(dim, target_level) * y

def dFdu(u, m):
    return dPdu(u, m)

In [ ]:
state_0 = qt.basis(dim, 0) #initial state

In [ ]:
u = np.zeros(f1.nu + f2.nu)
u[:f1.nu] = u1[:]
u[f1.nu:] = u2[:]

## Comparison, Schrödinger picture

In [ ]:
tg = target.Target('generic', Fyu = Fyu, dFdy = dFdy, dFdu = dFdu)

opt = qoct.Qoct(H, T, time.shape[0], tg, [f1, f2], state_0,
                interaction_picture = False, solve_method = 'cfmagnus4')

In [ ]:
u = np.zeros(f1.nu + f2.nu)
u[:f1.nu] = u1[:]
u[f1.nu:] = u2[:]

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

In [ ]:
derqoct, dernum, error = opt.check_grad(u, f1.nu + 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

## Comparison, interaction picture

In [ ]:
opt.interaction_picture = True

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

In [ ]:
derqoct, dernum, error = opt.check_grad(u, f1.nu + 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

# Gradient calculation for state-to-state transitions: parametrization with an auxiliary function

In [ ]:
def g(f, j):
    return f[0]

def gradg(f, j, l):
    return 1.0

In [ ]:
H = hamiltonians.hamiltonian(H0, [V1, V2], g = g, gradg = gradg)

In [ ]:
tg = target.Target('expectationvalue', operator = qt.fock_dm(dim, target_level))

opt = qoct.Qoct(H, T, time.shape[0], tg, [f1], state_0,
                interaction_picture = False,
                solve_method = 'cfmagnus4',
                new_parametrization = True)

In [ ]:
u = np.zeros(f1.nu)
u[:f1.nu] = u1[:]
#u[f1.nu:] = u2[:]

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

# Gradient calculation for state-to-state transitions: real-time parametrization of the pulse

In [ ]:
u1t = f1.fu(time)
u2t = f2.fu(time)
f1t = pulses.pulse("realtime", T, u = u1t)
f2t = pulses.pulse("realtime", T, u = u2t)

## QOCT target function definition

In [ ]:
state_0 = qt.basis(dim, 0) #initial state

In [ ]:
ut = np.zeros(f1t.nu + f2t.nu)
ut[:f1t.nu] = u1t[:]
ut[f1t.nu:] = u2t[:]

## Comparison, Schrödinger picture

In [ ]:
H = hamiltonians.hamiltonian(H0, [V1, V2])

In [ ]:
tg = target.Target('expectationvalue', operator = qt.fock_dm(dim, target_level))

opt = qoct.Qoct(H, T, time.shape[0], tg, [f1t, f2t], state_0,
                interaction_picture = False, solve_method = 'cfmagnus4')

In [ ]:
ut = np.zeros(f1t.nu + f2t.nu)
ut[:f1t.nu] = u1t[:]
ut[f2t.nu:f1t.nu+f2t.nu] = u2t[:]

In [ ]:
derqoct, dernum, error = opt.check_grad(ut, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

In [ ]:
with open("data", "w") as datafile:
    for i in data:
        datafile.write("{:.14e}\n".format(i))